In [36]:
import numpy as np

# Define the arrays for treasure and hunters
treasure = np.array([
            24, 70, 41, 21, 60, 
            47, 82, 87, 80, 35, 
            73, 89, 100, 90, 17, 
            77, 83, 85, 79, 55,
            12, 27, 52, 15, 30])

hunters = np.array([
            2, 4, 3, 2, 4, 
            3, 5, 5, 5, 3, 
            4, 5, 8, 7, 2, 
            5, 5, 5, 5, 4,
            2, 3, 4, 2, 3])

#treasure = np.array([24,70,47,82])
#hunters = np.array([2,4,3,5])

# Calculate the number of elements in the hunters array
n = len(hunters)

# Initialize the array y with zeros
y = np.zeros(n)

# Assign 100 to the last element of y
y[-1] = 100

# Loop to calculate differences as per the given formula
for i in range(n - 1):
    y[i] = hunters[i+1] / (treasure[i+1] * 7500) - hunters[i] / (treasure[i] * 7500)

# Print the resulting array y
#print(y)

A = np.zeros((n, n))

# Loop to populate the matrix for the diagonal and off-diagonal elements
for i in range(n - 1):
    A[i][i] = 1 / (treasure[i] * 7500)
    A[i][i+1] = -1 / (treasure[i+1] * 7500)

# Set the last row to all ones
A[n-1] = np.ones(n)

# Print the resulting matrix A
#print(A)

In [37]:
import numpy as np
from scipy.optimize import linprog

# Linear programming does not directly support equality constraints.
# Thus, we must separate A into two halves to express them as two inequality sets.
# We will minimize a zero function (as we're only interested in finding a feasible x).
c = np.zeros(n)  # Coefficients of the dummy objective function

# Bounds for each x_i: 0 < x_i < 1
x_bounds = [(0, 100) for _ in range(n)]

# Define constraints (Ax = y converted into Ax <= y and Ax >= y)
res = linprog(c, A_eq=A, b_eq=y, bounds=x_bounds, method='highs')

if res.success:
    print("Solution x:", res.x)
else:
    print("No solution found:", res.message)


No solution found: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)


In [53]:
import numpy as np

# Try solving using least squares to see if a solution is fundamentally possible
x_ls, residuals, rank, s = np.linalg.lstsq(A, y, rcond=None)

# Check if the solution x_ls lies within the desired bounds
solution_within_bounds = np.all((x_ls > 0) & (x_ls < 100))

print("Least Squares Solution:", x_ls)
print("Solution within bounds:", solution_within_bounds)


Least Squares Solution: [ 1.35429769  5.78336827  2.73025856  0.93501048  4.38574423  3.56883298
  6.46051712  7.15932914  6.18099231  1.89168414  6.20265549  7.43885395
  5.97624039  5.57861635  0.37596087  5.7617051   6.60027952  6.87980433
  6.04122991  3.68693222 -0.32285115  0.77358491  3.267645    0.09643606
  1.19287212]
Solution within bounds: False


In [56]:
for i in range(n):
    x_ls[i] = int(round(x_ls[i]))
print(x_ls.reshape((5, 5)))

[[1. 6. 3. 1. 4.]
 [4. 6. 7. 6. 2.]
 [6. 7. 6. 6. 0.]
 [6. 7. 7. 6. 4.]
 [0. 1. 3. 0. 1.]]
